In [2]:
# Usage
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM

tokenizer = LlamaTokenizer.from_pretrained('sarvamai/OpenHathi-7B-Hi-v0.1-Base')
model = LlamaForCausalLM.from_pretrained('sarvamai/OpenHathi-7B-Hi-v0.1-Base', torch_dtype=torch.bfloat16)

prompt = "मैं एक अच्छा हाथी हूँ"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

tokenizer_config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/968k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/3.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Keyword arguments {'add_special_tokens': False} not recognized.


"मैं एक अच्छा हाथी हूँ।\n\nI'm a good elephant, I'm a good elephant,"

In [3]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 48.5 MB/s eta 0:00:00:00:0100:01


In [4]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM
import speech_recognition as sr
from pydub import AudioSegment

# Initialize tokenizer and model
tokenizer = LlamaTokenizer.from_pretrained('sarvamai/OpenHathi-7B-Hi-v0.1-Base')
model = LlamaForCausalLM.from_pretrained('sarvamai/OpenHathi-7B-Hi-v0.1-Base', torch_dtype=torch.bfloat16)

# Initialize the recognizer
recognizer = sr.Recognizer()

# Load the MP3 audio file
mp3_file = "/kaggle/input/indian-languages-audio-dataset/Indian_Languages_Audio_Dataset/Hindi/10304.mp3"
wav_file = "/kaggle/working/audio.wav"

# Convert MP3 to WAV
audio = AudioSegment.from_mp3(mp3_file)
audio.export(wav_file, format="wav")

# Open the WAV audio file
with sr.AudioFile(wav_file) as source:
    # Record the audio
    audio_data = recognizer.record(source)

# Convert speech to text
try:
    transcript = recognizer.recognize_google(audio_data, language="hi-IN")  # Specify the language if needed
    print("Transcription: ", transcript)
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand the audio")
except sr.RequestError as e:
    print("Could not request results from Google Speech Recognition service; {0}".format(e))

# Now, use the transcribed text as the prompt
prompt = transcript
inputs = tokenizer(prompt, return_tensors="pt")

# Generate text based on the transcribed text
generate_ids = model.generate(inputs.input_ids, max_length=30)
generated_text = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

print("Generated Text: ", generated_text)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Keyword arguments {'add_special_tokens': False} not recognized.


Transcription:  नामक एक स्थानीय पेशेवर बास्केटबॉल टीम के अगले प्रदर्शन
Generated Text:  नामक एक स्थानीय पेशेवर बास्केटबॉल टीम के अगले प्रदर्शन के लिए एक विज्ञापन के लिए एक मॉडल के रूप में काम किया


In [5]:
!pip install translate

In [6]:
!pip install git+https://github.com/openai/whisper.git -q

In [7]:
import whisper
model = whisper.load_model("large")

100%|██████████████████████████████████████| 2.88G/2.88G [00:22<00:00, 136MiB/s]


In [20]:
audio=whisper.load_audio("/kaggle/input/indian-languages-audio-dataset/Indian_Languages_Audio_Dataset/Hindi/10304.mp3")
audio = whisper.pad_or_trim(audio) 

In [21]:
# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio, n_mels=128).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)
     

Detected language: hi
नामक एक इस्थानीय पेशेवर बास्केटबॉल टीम के अगले पर्दसन के लिए


In [22]:
results=result.text

In [25]:
import pandas as pd
# Create a DataFrame
df = pd.DataFrame({
    "OpenHathi": generated_text,
    "WhisperAI": results
},index=[0, 1])

df

,OpenHathi,WhisperAI
0,नामक एक स्थानीय पेशेवर बास्केटबॉल टीम के अगले ...,नामक एक इस्थानीय पेशेवर बास्केटबॉल टीम के अगले...
1,नामक एक स्थानीय पेशेवर बास्केटबॉल टीम के अगले ...,नामक एक इस्थानीय पेशेवर बास्केटबॉल टीम के अगले...


In [10]:
audio = whisper.load_audio("/kaggle/input/indian-languages-audio-dataset/Indian_Languages_Audio_Dataset/Hindi/10304.mp3")
options = {"fp16": False, "language": "hi", "task": "translate"}
result = model.transcribe(audio, **options)
print(result)

{'text': ' For the next match of the local basketball team named Hoopers', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 5.0, 'text': ' For the next match of the local basketball team named Hoopers', 'tokens': [50365, 1171, 264, 958, 2995, 295, 264, 2654, 11767, 1469, 4926, 3631, 33338, 50615], 'temperature': 0.0, 'avg_logprob': -0.9067390441894532, 'compression_ratio': 0.9682539682539683, 'no_speech_prob': 0.28305506706237793}], 'language': 'hi'}


In [3]:
import os
import pandas as pd
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM
import speech_recognition as sr
from pydub import AudioSegment

# Initialize tokenizer and model
tokenizer = LlamaTokenizer.from_pretrained('sarvamai/OpenHathi-7B-Hi-v0.1-Base')
model = LlamaForCausalLM.from_pretrained('sarvamai/OpenHathi-7B-Hi-v0.1-Base', torch_dtype=torch.bfloat16)

# Initialize the recognizer
recognizer = sr.Recognizer()

# Directory containing audio files
mp3_directory = "/kaggle/input/indian-languages-audio-dataset/Indian_Languages_Audio_Dataset/Hindi"
wav_file = "/kaggle/working/audio.wav"

# List to store transcriptions
transcriptions = []

# Loop through each file in the directory
count = 0
for filename in os.listdir(mp3_directory):
    if filename.endswith(".mp3"):
        mp3_file = os.path.join(mp3_directory, filename)

        # Convert MP3 to WAV
        audio = AudioSegment.from_mp3(mp3_file)
        audio.export(wav_file, format="wav")

        # Open the WAV audio file
        with sr.AudioFile(wav_file) as source:
            # Record the audio
            audio_data = recognizer.record(source)

        # Convert speech to text
        try:
            transcript = recognizer.recognize_google(audio_data, language="hi-IN")  
            transcriptions.append(transcript)
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
            continue
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))
            continue

        count += 1
        if count == 100:
            break

# Create a DataFrame to store the transcriptions
df1 = pd.DataFrame({"Transcription": transcriptions})

# Save the DataFrame to a CSV file
df1.to_csv("/kaggle/working/transcriptions.csv", index=False)

print("Transcriptions saved to transcriptions.csv")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Transcriptions saved to transcriptions.csv


In [12]:
import os
import pandas as pd
import whisper
from pydub import AudioSegment

# Load the whisper model
model = whisper.load_model("large")

# Directory containing audio files
mp3_directory = "/kaggle/input/indian-languages-audio-dataset/Indian_Languages_Audio_Dataset/Hindi"

# List to store transcriptions and decoded texts
transcriptions = []
decoded_texts = []

# Loop through each file in the directory
count = 0
for filename in os.listdir(mp3_directory):
    if filename.endswith(".mp3"):
        mp3_file = os.path.join(mp3_directory, filename)

        # Pad or trim audio
        audio = whisper.load_audio(mp3_file)
        audio = whisper.pad_or_trim(audio)

        # Make log-Mel spectrogram and move to the same device as the model
        mel = whisper.log_mel_spectrogram(audio, n_mels=128).to(model.device)

        # Decode the audio
        options = whisper.DecodingOptions()
        result1 = whisper.decode(model, mel, options)

        # Store transcription and decoded text
        transcriptions.append(result1.text)
        decoded_texts.append(result1.text)

        count += 1
        if count == 100:
            break

# Create a DataFrame to store transcriptions and decoded texts
df1 = pd.DataFrame({"Transcriptions": transcriptions, "Decoded_Text": decoded_texts})

# Print the DataFrame
print(df1)

# Save the DataFrame to a CSV file
df1.to_csv("/kaggle/working/transcriptions_and_decoded_texts.csv", index=False)

print("Transcriptions and decoded texts saved to transcriptions_and_decoded_texts.csv")


                                        Transcription  \
0   पर मुझे उसे पुकार कर कुछ पूछने की हिम्मत नहीं ...   
1   इनकी किमत कितनी है और वे उन्हें कहां से खरीद स...   
2   अकाउंटेंट के बुनियादी कामों में से एक यह है कि...   
3                   पूरे कारवार का मालिक होने पर करता   
4   उतना ही बहतर है। इसलिए, referrals इतने शक्तिशा...   
..                                                ...   
95  अपने परिवार के साथ ओ महा गया तो महच 14 साल के ...   
96           करते हुए वारन ने ओमहा के चार्ल्स हीडर से   
97  प्रश्नास का संयुक्त राज्य अमेरिका संप्रति वर्क...   
98  ऐसा सांधार नतीजा सामने आया लेकिन इस तथ्य से कि...   
99  कि वे शारीरिक शक्ति, विचार या ध्यान के प्रभाव ...   

                                         Decoded_Text  
0   पर मुझे उसे पुकार कर कुछ पूछने की हिम्मत नहीं ...  
1   इनकी किमत कितनी है और वे उन्हें कहां से खरीद स...  
2   अकाउंटेंट के बुनियादी कामों में से एक यह है कि...  
3                   पूरे कारवार का मालिक होने पर करता  
4   उतना ही बहतर है। इसलिए, referra

In [28]:
df1 = pd.DataFrame({"Open_Hathi": transcriptions, "Whisper_AI": decoded_texts})
df1.head()

,Open_Hathi,Whisper_AI
0,पर मुझे उसे पुकार कर कुछ पूछने की हिम्मत नहीं ...,पर मुझे उसे पुकार कर कुछ पूछने की हिम्मत नहीं ...
1,इनकी किमत कितनी है और वे उन्हें कहां से खरीद स...,इनकी किमत कितनी है और वे उन्हें कहां से खरीद स...
2,अकाउंटेंट के बुनियादी कामों में से एक यह है कि...,अकाउंटेंट के बुनियादी कामों में से एक यह है कि...
3,पूरे कारवार का मालिक होने पर करता,पूरे कारवार का मालिक होने पर करता
4,"उतना ही बहतर है। इसलिए, referrals इतने शक्तिशा...","उतना ही बहतर है। इसलिए, referrals इतने शक्तिशा..."


In [16]:
!pip install jiwer

In [32]:
import pandas as pd
import jiwer

# Calculate WER for each column
wer_open_hathi = jiwer.wer(list(df1['Open_Hathi']), list(df1['Whisper_AI']))
wer_whisper_ai = jiwer.wer(list(df1['Whisper_AI']), list(df1['Open_Hathi']))  
print(f"WER for Open_Hathi: {wer_open_hathi * 100:.2f} %")
print(f"WER for Whisper_AI: {wer_whisper_ai * 100:.2f} %")


WER for Open_Hathi: 0.00 %
WER for Whisper_AI: 0.00 %
